In [21]:
!pip install --upgrade pyspark_dist_explore

Requirement not upgraded as not directly required: numpy in /opt/anaconda/lib/python3.6/site-packages (from pyspark_dist_explore) (1.15.0)
Requirement not upgraded as not directly required: matplotlib in /opt/anaconda/lib/python3.6/site-packages (from pyspark_dist_explore) (2.2.2)
Requirement not upgraded as not directly required: pandas in /opt/anaconda/lib/python3.6/site-packages (from pyspark_dist_explore) (0.23.3)
Requirement not upgraded as not directly required: scipy in /opt/anaconda/lib/python3.6/site-packages (from pyspark_dist_explore) (1.1.0)
Requirement not upgraded as not directly required: cycler>=0.10 in /opt/anaconda/lib/python3.6/site-packages (from matplotlib->pyspark_dist_explore) (0.10.0)
Requirement not upgraded as not directly required: pyparsing!=2.0.4,!=2.1.2,!=2.1.6,>=2.0.1 in /opt/anaconda/lib/python3.6/site-packages (from matplotlib->pyspark_dist_explore) (2.2.0)
Requirement not upgraded as not directly required: python-dateutil>=2.1 in /opt/anaconda/lib/pyth

In [38]:
# imports
import re
import ast
import time
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
import os
from IPython.display import display, HTML, display_html #usefull to display wide tables
from pyspark_dist_explore import Histogram, hist, distplot, pandas_histogram
from pyspark.sql import functions as F, types, SQLContext
from pyspark.ml.feature import OneHotEncoder, StringIndexer

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
# store path to notebook
PWD = !pwd
PWD = PWD[0]

In [4]:
fifty_lines = ['0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16',
 '0\t2\t0\t44\t1\t102\t8\t2\t2\t4\t1\t1\t\t4\t68fd1e64\tf0cf0024\t6f67f7e5\t41274cd7\t25c83c98\tfe6b92e5\t922afcc0\t0b153874\ta73ee510\t2b53e5fb\t4f1b46f3\t623049e6\td7020589\tb28479f6\te6c5b5cd\tc92f3b61\t07c540c4\tb04e4670\t21ddcdc9\t5840adea\t60f6221e\t\t3a171ecb\t43f13e8b\te8b83407\t731c3655',
 '0\t2\t0\t1\t14\t767\t89\t4\t2\t245\t1\t3\t3\t45\t287e684f\t0a519c5c\t02cf9876\tc18be181\t25c83c98\t7e0ccccf\tc78204a1\t0b153874\ta73ee510\t3b08e48b\t5f5e6091\t8fe001f4\taa655a2f\t07d13a8f\t6dc710ed\t36103458\t8efede7f\t3412118d\t\t\te587c466\tad3062eb\t3a171ecb\t3b183c5c\t\t',
 '0\t\t893\t\t\t4392\t\t0\t0\t0\t\t0\t\t\t68fd1e64\t2c16a946\ta9a87e68\t2e17d6f6\t25c83c98\tfe6b92e5\t2e8a689b\t0b153874\ta73ee510\tefea433b\te51ddf94\ta30567ca\t3516f6e6\t07d13a8f\t18231224\t52b8680f\t1e88c74f\t74ef3502\t\t\t6b3a5ca6\t\t3a171ecb\t9117a34a\t\t',
 '0\t3\t-1\t\t0\t2\t0\t3\t0\t0\t1\t1\t\t0\t8cf07265\tae46a29d\tc81688bb\tf922efad\t25c83c98\t13718bbd\tad9fa255\t0b153874\ta73ee510\t5282c137\te5d8af57\t66a76a26\tf06c53ac\t1adce6ef\t8ff4b403\t01adbab4\t1e88c74f\t26b3c7a7\t\t\t21c9516a\t\t32c7478e\tb34f3128\t\t']

In [5]:
from pyspark.sql import SparkSession
app_name = "hw5_notebook"
master = "local[*]"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .master(master)\
        .getOrCreate()
sc = spark.sparkContext

In [6]:
trdd = sc.parallelize(fifty_lines)

In [7]:
trdd.collect()

['0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16',
 '0\t2\t0\t44\t1\t102\t8\t2\t2\t4\t1\t1\t\t4\t68fd1e64\tf0cf0024\t6f67f7e5\t41274cd7\t25c83c98\tfe6b92e5\t922afcc0\t0b153874\ta73ee510\t2b53e5fb\t4f1b46f3\t623049e6\td7020589\tb28479f6\te6c5b5cd\tc92f3b61\t07c540c4\tb04e4670\t21ddcdc9\t5840adea\t60f6221e\t\t3a171ecb\t43f13e8b\te8b83407\t731c3655',
 '0\t2\t0\t1\t14\t767\t89\t4\t2\t245\t1\t3\t3\t45\t287e684f\t0a519c5c\t02cf9876\tc18be181\t25c83c98\t7e0ccccf\tc78204a1\t0b153874\ta73ee510\t3b08e48b\t5f5e6091\t8fe001f4\taa655a2f\t07d13a8f\t6dc710ed\t36103458\t8efede7f\t3412118d\t\t\te587c466\tad3062eb\t3a171ecb\t3b183c5c\t\t',
 '0\t\t893\t\t\t4392\t\t0\t0\t0\t\t0\t\t\t68fd1e64\t2c16a946\ta9a87e68\t2e17d6f6\t25c83c98\tfe6b92e5\t2e8a689b\t0b15387

In [8]:
trdd.take(1)

['0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16']

In [9]:
trdd.take(2)

['0\t1\t1\t5\t0\t1382\t4\t15\t2\t181\t1\t2\t\t2\t68fd1e64\t80e26c9b\tfb936136\t7b4723c4\t25c83c98\t7e0ccccf\tde7995b8\t1f89b562\ta73ee510\ta8cd5504\tb2cb9c98\t37c9c164\t2824a5f6\t1adce6ef\t8ba8b39a\t891b62e7\te5ba7672\tf54016b9\t21ddcdc9\tb1252a9d\t07b5194c\t\t3a171ecb\tc5c50484\te8b83407\t9727dd16',
 '0\t2\t0\t44\t1\t102\t8\t2\t2\t4\t1\t1\t\t4\t68fd1e64\tf0cf0024\t6f67f7e5\t41274cd7\t25c83c98\tfe6b92e5\t922afcc0\t0b153874\ta73ee510\t2b53e5fb\t4f1b46f3\t623049e6\td7020589\tb28479f6\te6c5b5cd\tc92f3b61\t07c540c4\tb04e4670\t21ddcdc9\t5840adea\t60f6221e\t\t3a171ecb\t43f13e8b\te8b83407\t731c3655']

In [10]:
students_dict = {
   "Andrew" : [1, 4, 8, 2.9, 1, "Computer Science", "History"],
   "Vikram" : [0, 3, 9, 3.5, 0, "Math", "French"],
   "Samantha" : [0, 2, 9, 3.4, 1, "Math", "Computer Science"],
   "Chris" : [0, 2, 8, 3.7, 0, "Data Science", "English"],
   "Brenda" : [0, 3, 7, 3.9, 0, "Data Science", "Math"],
   "Doug" : [1, 3, 7, 3.2, 0, "English", "Data Science"],
   "Maria" : [1, 4, 8, 2.9, 1, "Computer Science", "History"],
   "Tony" : [0, 2, 7, 2.8, 1, "Data Science", "French"],
   "Tonya" : [1, 4, 9, 3.7, 0, "Computer Science", "German"],
   "Ruth" : [0, 3, 6, 3.1, 1, "Astrophysics", "Math"]
}

In [109]:
for x in students_dict.values():
    print(','.join(map(str, x)))

1,4,8,2.9,1,Computer Science,History
0,3,9,3.5,0,Math,French
0,2,9,3.4,1,Math,Computer Science
0,2,8,3.7,0,Data Science,English
0,3,7,3.9,0,Data Science,Math
1,3,7,3.2,0,English,Data Science
1,4,8,2.9,1,Computer Science,History
0,2,7,2.8,1,Data Science,French
1,4,9,3.7,0,Computer Science,German
0,3,6,3.1,1,Astrophysics,Math


In [15]:
toyRDD = sc.parallelize(students_dict.values())

In [52]:
toyRDD1 = sc.parallelize([ ','.join(map(str, x)) for x in students_dict.values()])

In [18]:
toyDF = toyRDD.map(lambda x: x).toDF(["dropout", "int_1", "int_2", "int_3", "int_4", "categ_1", "categ_2"])

In [27]:
toyDF = toyDF.withColumn("int_1", toyDF["int_1"].cast(types.IntegerType()))
toyDF = toyDF.withColumn("int_2", toyDF["int_2"].cast(types.IntegerType()))
toyDF = toyDF.withColumn("int_3", toyDF["int_3"].cast(types.IntegerType()))
toyDF = toyDF.withColumn("int_4", toyDF["int_4"].cast(types.IntegerType()))
toyDF = toyDF.withColumn("dropout", toyDF["int_4"].cast(types.IntegerType()))

In [28]:
toyDF.head()

Row(dropout=1, int_1=4, int_2=8, int_3=2, int_4=1, categ_1='Computer Science', categ_2='History')

In [32]:
from collections import defaultdict
# Data for manual OHE
# Note: the first data point does not include any value for the optional third feature
sample_one = [(0, 'mouse'), (1, 'black')]
sample_two = [(0, 'cat'), (1, 'tabby'), (2, 'mouse')]
sample_three =  [(0, 'bear'), (1, 'black'), (2, 'salmon')]

def sample_to_row(sample):
    tmp_dict = defaultdict(lambda: None)
    tmp_dict.update(sample)
    return [tmp_dict[i] for i in range(4)]

In [29]:
toyDF.show()

+-------+-----+-----+-----+-----+----------------+----------------+
|dropout|int_1|int_2|int_3|int_4|         categ_1|         categ_2|
+-------+-----+-----+-----+-----+----------------+----------------+
|      1|    4|    8|    2|    1|Computer Science|         History|
|      0|    3|    9|    3|    0|            Math|          French|
|      1|    2|    9|    3|    1|            Math|Computer Science|
|      0|    2|    8|    3|    0|    Data Science|         English|
|      0|    3|    7|    3|    0|    Data Science|            Math|
|      0|    3|    7|    3|    0|         English|    Data Science|
|      1|    4|    8|    2|    1|Computer Science|         History|
|      1|    2|    7|    2|    1|    Data Science|          French|
|      0|    4|    9|    3|    0|Computer Science|          German|
|      1|    3|    6|    3|    1|    Astrophysics|            Math|
+-------+-----+-----+-----+-----+----------------+----------------+



In [110]:
%%writefile toy.txt
1,4,8,2.9,1,Computer Science,History
0,3,9,3.5,0,Math,French
0,2,9,3.4,1,Math,Computer Science
0,2,8,3.7,0,Data Science,English
0,3,7,3.9,0,Data Science,Math
1,3,7,3.2,0,English,Data Science
1,4,8,2.9,1,Computer Science,History
0,2,7,2.8,1,Data Science,French
1,4,9,3.7,0,Computer Science,German
0,3,6,3.1,1,Astrophysics,Math

Writing toy.txt


In [34]:
sample_to_row(sample_one)

['mouse', 'black', None, None]

In [35]:
toyDF["categ_1"].value_counts()

TypeError: 'Column' object is not callable

In [55]:
raw_df = sqlContext.createDataFrame(toyRDD1).withColumnRenamed("value", "text")

TypeError: Can not infer schema for type: <class 'str'>

In [41]:
sqlContext = SQLContext(sc)

In [44]:
raw_df.show()

+---+---+---+---+---+----------------+----------------+
| _1| _2| _3| _4| _5|              _6|              _7|
+---+---+---+---+---+----------------+----------------+
|  1|  4|  8|2.9|  1|Computer Science|         History|
|  0|  3|  9|3.5|  0|            Math|          French|
|  0|  2|  9|3.4|  1|            Math|Computer Science|
|  0|  2|  8|3.7|  0|    Data Science|         English|
|  0|  3|  7|3.9|  0|    Data Science|            Math|
|  1|  3|  7|3.2|  0|         English|    Data Science|
|  1|  4|  8|2.9|  1|Computer Science|         History|
|  0|  2|  7|2.8|  1|    Data Science|          French|
|  1|  4|  9|3.7|  0|Computer Science|          German|
|  0|  3|  6|3.1|  1|    Astrophysics|            Math|
+---+---+---+---+---+----------------+----------------+



In [54]:
toyRDD1.collect()

['1,4,8,2.9,1,Computer Science,History',
 '0,3,9,3.5,0,Math,French',
 '0,2,9,3.4,1,Math,Computer Science',
 '0,2,8,3.7,0,Data Science,English',
 '0,3,7,3.9,0,Data Science,Math',
 '1,3,7,3.2,0,English,Data Science',
 '1,4,8,2.9,1,Computer Science,History',
 '0,2,7,2.8,1,Data Science,French',
 '1,4,9,3.7,0,Computer Science,German',
 '0,3,6,3.1,1,Astrophysics,Math']

In [97]:
df = pd.DataFrame(list(students_dict.values()), columns=["dropout", "int_1", "int_2", "int_3", "int_4", "categ_1", "categ_2"])

In [98]:
df.describe()

,dropout,int_1,int_2,int_3,int_4
count,10.000000,10.000000,10.000000,10.000000,10.000000
mean,0.400000,3.000000,7.800000,3.310000,0.500000
std,0.516398,0.816497,1.032796,0.387155,0.527046
min,0.000000,2.000000,6.000000,2.800000,0.000000
25%,0.000000,2.250000,7.000000,2.950000,0.000000
50%,0.000000,3.000000,8.000000,3.300000,0.500000
75%,1.000000,3.750000,8.750000,3.650000,1.000000
max,1.000000,4.000000,9.000000,3.900000,1.000000


In [101]:
df

,dropout,int_1,int_2,int_3,int_4,categ_1,categ_2
0,1,4,8,2.9,1,Computer Science,History
1,0,3,9,3.5,0,Math,French
2,0,2,9,3.4,1,Math,Computer Science
3,0,2,8,3.7,0,Data Science,English
4,0,3,7,3.9,0,Data Science,Math
5,1,3,7,3.2,0,English,Data Science
6,1,4,8,2.9,1,Computer Science,History
7,0,2,7,2.8,1,Data Science,French
8,1,4,9,3.7,0,Computer Science,German
9,0,3,6,3.1,1,Astrophysics,Math


In [92]:
list(df['categ_1'].unique())

['Computer Science', 'Math', 'Data Science', 'English', 'Astrophysics']

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 7 columns):
dropout    10 non-null int64
int_1      10 non-null int64
int_2      10 non-null int64
int_3      10 non-null float64
int_4      10 non-null int64
categ_1    10 non-null object
categ_2    10 non-null object
dtypes: float64(1), int64(4), object(2)
memory usage: 640.0+ bytes


In [102]:
df['categ_1'] = pd.Categorical(df['categ_1'], categories = list(df['categ_1'].unique())+['unk'])
df['categ_2'] = pd.Categorical(df['categ_2'], categories = list(df['categ_2'].unique())+['unk'])

In [103]:
df = pd.get_dummies(df, columns=['categ_1', 'categ_2'], prefix = ['categ_1', 'categ_2'])

In [104]:
df

,dropout,int_1,int_2,int_3,int_4,categ_1_Computer Science,categ_1_Math,categ_1_Data Science,categ_1_English,categ_1_Astrophysics,categ_1_unk,categ_2_History,categ_2_French,categ_2_Computer Science,categ_2_English,categ_2_Math,categ_2_Data Science,categ_2_German,categ_2_unk
0,1,4,8,2.9,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
1,0,3,9,3.5,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0
2,0,2,9,3.4,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
3,0,2,8,3.7,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0
4,0,3,7,3.9,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0
5,1,3,7,3.2,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
6,1,4,8,2.9,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0
7,0,2,7,2.8,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0
8,1,4,9,3.7,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0
9,0,3,6,3.1,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0


In [105]:
toySPDF = sqlContext.createDataFrame(df)

In [106]:
toySPDF.show()

+-------+-----+-----+-----+-----+------------------------+------------+--------------------+---------------+--------------------+-----------+---------------+--------------+------------------------+---------------+------------+--------------------+--------------+-----------+
|dropout|int_1|int_2|int_3|int_4|categ_1_Computer Science|categ_1_Math|categ_1_Data Science|categ_1_English|categ_1_Astrophysics|categ_1_unk|categ_2_History|categ_2_French|categ_2_Computer Science|categ_2_English|categ_2_Math|categ_2_Data Science|categ_2_German|categ_2_unk|
+-------+-----+-----+-----+-----+------------------------+------------+--------------------+---------------+--------------------+-----------+---------------+--------------+------------------------+---------------+------------+--------------------+--------------+-----------+
|      1|    4|    8|  2.9|    1|                       1|           0|                   0|              0|                   0|          0|              1|             0|   

In [113]:
toy_rawDF = sqlContext.read.text('toy.txt').withColumnRenamed("value", "text")


In [114]:
toy_rawDF.show()

+--------------------+
|                text|
+--------------------+
|1,4,8,2.9,1,Compu...|
|0,3,9,3.5,0,Math,...|
|0,2,9,3.4,1,Math,...|
|0,2,8,3.7,0,Data ...|
|0,3,7,3.9,0,Data ...|
|1,3,7,3.2,0,Engli...|
|1,4,8,2.9,1,Compu...|
|0,2,7,2.8,1,Data ...|
|1,4,9,3.7,0,Compu...|
|0,3,6,3.1,1,Astro...|
+--------------------+



In [ ]:
labelColList=["clicked_0_1"]
intColList=["int_1", "int_2", "int_3", "int_4", "int_5", "int_6", "int_7", "int_8", "int_9", "int_10", "int_11", "int_12", "int_13"]
categColList=["categ_1", "categ_2"

In [116]:
weights = [.8, .1, .1]
seed = 42
# Use randomSplit with weights and seed
raw_train_df, raw_validation_df, raw_test_df = toy_rawDF.randomSplit(weights, seed)

# Cache and count the DataFrames
n_train = raw_train_df.cache().count()
n_val = raw_validation_df.cache().count()
n_test = raw_test_df.cache().count()
print(n_train, n_val, n_test, str(n_train + n_val + n_test))


7 2 1 10


In [121]:
# TODO: Replace <FILL IN> with appropriate code
def parse_point(point):
    """Converts a comma separated string into a list of (featureID, value) tuples.

    Note:
        featureIDs should start at 0 and increase to the number of features - 1.

    Args:
        point (str): A comma separated string where the first value is the label and the rest
            are features.

    Returns:
        list: A list of (featureID, value) tuples.
    """
    values = point.split(',')[1:]
    #values = filter(None, values)
    indices = range(len(values))
    return zip(indices,values)

print(list(parse_point(toy_rawDF.select('text').first()[0])))

[(0, '4'), (1, '8'), (2, '2.9'), (3, '1'), (4, 'Computer Science'), (5, 'History')]


In [119]:
toy_rawDF.select('text').first()[0]

'1,4,8,2.9,1,Computer Science,History'

In [120]:
parse_point(toy_rawDF.select('text').first()[0])

In [184]:
from pyspark.sql.functions import udf, split
from pyspark.sql.types import ArrayType, StructType, StructField, LongType, StringType, FloatType, DoubleType


parse_point_udf = udf(parse_point, ArrayType(StructType([StructField('_1', LongType()),StructField('_2', StringType())])))

def parse_raw_df(raw_df):
    """Convert a DataFrame consisting of rows of comma separated text into labels and feature.


    Args:
        raw_df (DataFrame with a 'text' column): DataFrame containing the raw comma separated data.

    Returns:
        DataFrame: A DataFrame with 'label' and 'feature' columns.   
  
    """
    return (raw_df.select(split(raw_df.text,',').getItem(0).cast("double").alias('label'),
                         parse_point_udf(raw_df.text).alias('features'))
                        .cache())

In [194]:
parse_point_udf1 = udf(parse_point, ArrayType(StructType([StructField('_1', LongType()), StructField('_2', FloatType()), StructField('_3', FloatType()), StructField('_4', FloatType()), StructField('_5', FloatType()), StructField('_6', StringType())])) )

In [195]:
def parse_raw_df1(raw_df):
    """Convert a DataFrame consisting of rows of comma separated text into labels and feature.


    Args:
        raw_df (DataFrame with a 'text' column): DataFrame containing the raw comma separated data.

    Returns:
        DataFrame: A DataFrame with 'label' and 'feature' columns.   
  
    """
    return (raw_df.select(split(raw_df.text,',').getItem(0).cast("double").alias('label'),
                         parse_point_udf1(raw_df.text).alias('features'))
                        .cache())

In [189]:
parsed_train_df = parse_raw_df(raw_train_df)
print(parsed_train_df.head())

Row(label=0.0, features=[Row(_1=0, _2='2'), Row(_1=1, _2='7'), Row(_1=2, _2='2.8'), Row(_1=3, _2='1'), Row(_1=4, _2='Data Science'), Row(_1=5, _2='French')])


In [197]:
parsed_train_df1 = parse_raw_df1(raw_train_df)
print(parsed_train_df1.head())

Py4JJavaError: An error occurred while calling o1190.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 89.0 failed 1 times, most recent failure: Lost task 0.0 in stage 89.0 (TID 1498, localhost, executor driver): java.lang.IllegalStateException: Input row doesn't have expected number of values required by the schema. 6 fields are required while 2 values are provided.
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15$$anonfun$apply$15.applyOrElse(EvaluatePython.scala:184)
	at org.apache.spark.sql.execution.python.EvaluatePython$.org$apache$spark$sql$execution$python$EvaluatePython$$nullSafeConvert(EvaluatePython.scala:208)
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$15.apply(EvaluatePython.scala:180)
	at org.apache.spark.sql.execution.python.EvaluatePython$$anonfun$makeFromJava$13$$anonfun$apply$13$$anonfun$applyOrElse$1.apply(EvaluatePython.scala:160)
	at scala.collect

In [126]:
from pyspark.sql.functions import (explode, col)
num_categories = (parsed_train_df
                    .select(explode('features').alias('features'))
                    .distinct()
                    .select(col('features').getField('_1').alias('featureNumber'))
                    .groupBy('featureNumber')
                    .sum()
                    .orderBy('featureNumber')
                    .collect())

In [127]:
print(num_categories)

[Row(featureNumber=0, sum(featureNumber)=0), Row(featureNumber=1, sum(featureNumber)=3), Row(featureNumber=2, sum(featureNumber)=12), Row(featureNumber=3, sum(featureNumber)=6), Row(featureNumber=4, sum(featureNumber)=16), Row(featureNumber=5, sum(featureNumber)=25)]


In [136]:
from pyspark.mllib.linalg import SparseVector
def one_hot_encoding(raw_feats, ohe_dict_broadcast, num_ohe_feats):
    """Produce a one-hot-encoding from a list of features and an OHE dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted.

    Args:
        raw_feats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sample_one)
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.
        num_ohe_feats (int): The total number of unique OHE features (combinations of featureID and
            value).

    Returns:
        SparseVector: A SparseVector of length num_ohe_feats with indices equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    indices = sorted([ohe_dict_broadcast.value[feat] for feat in raw_feats])
    values = np.ones(len(raw_feats))
    return SparseVector(num_ohe_feats,indices,values)

In [132]:
from pyspark.sql.functions import udf
from pyspark.mllib.linalg import VectorUDT

def ohe_udf_generator(ohe_dict_broadcast):
    """Generate a UDF that is setup to one-hot-encode rows with the given dictionary.

    Note:
        We'll reuse this function to generate a UDF that can one-hot-encode rows based on a
        one-hot-encoding dictionary built from the training data.  Also, you should calculate
        the number of features before calling the one_hot_encoding function.

    Args:
        ohe_dict_broadcast (Broadcast of dict): Broadcast variable containing a dict that maps
            (featureID, value) to unique integer.

    Returns:
        UserDefinedFunction: A UDF can be used in `DataFrame` `select` statement to call a
            function on each row in a given column.  This UDF should call the one_hot_encoding
            function with the appropriate parameters.
    """
    length = len(ohe_dict_broadcast.value)
    return udf(lambda x: one_hot_encoding(x, ohe_dict_broadcast, length), VectorUDT())


In [128]:
# TODO: Replace <FILL IN> with appropriate code
def create_one_hot_dict(input_df):
    """Creates a one-hot-encoder dictionary based on the input data.

    Args:
        input_df (DataFrame with 'features' column): A DataFrame where each row contains a list of
            (featureID, value) tuples.

    Returns:
        dict: A dictionary where the keys are (featureID, value) tuples and map to values that are
            unique integers.
    """
    input_distinct_feats_df = input_df.select(explode(input_df.features)).distinct()
    input_ohe_dict = (input_distinct_feats_df
                     .rdd
                     .map(lambda r: tuple(r[0]))
                     .zipWithIndex().collectAsMap())
    return input_ohe_dict


In [130]:
ctr_ohe_dict = create_one_hot_dict(parsed_train_df)
num_ctr_ohe_feats = len(ctr_ohe_dict)
print(num_ctr_ohe_feats)
ctr_ohe_dict

23


{(1, '9'): 0,
 (3, '1'): 1,
 (4, 'Math'): 2,
 (4, 'Data Science'): 3,
 (5, 'History'): 4,
 (0, '4'): 5,
 (2, '3.9'): 6,
 (5, 'Data Science'): 7,
 (5, 'French'): 8,
 (2, '3.5'): 9,
 (3, '0'): 10,
 (2, '2.9'): 11,
 (0, '3'): 12,
 (5, 'German'): 13,
 (5, 'Math'): 14,
 (1, '7'): 15,
 (1, '8'): 16,
 (4, 'English'): 17,
 (4, 'Computer Science'): 18,
 (2, '2.8'): 19,
 (2, '3.7'): 20,
 (2, '3.2'): 21,
 (0, '2'): 22}

In [139]:
ohe_dict_broadcast = sc.broadcast(ctr_ohe_dict)
ohe_dict_udf =  ohe_udf_generator(ohe_dict_broadcast)
ohe_train_df =  parsed_train_df.select(parsed_train_df.label, ohe_dict_udf(parsed_train_df.features).alias('features'))
#ohe_train_df.show(1)                  

print(ohe_train_df.count())
print(ohe_train_df.show())
print(ohe_train_df.take(1))

7
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(23,[1,3,8,15,19,...|
|  0.0|(23,[3,6,10,12,14...|
|  0.0|(23,[0,2,8,9,10,1...|
|  1.0|(23,[7,10,12,15,1...|
|  1.0|(23,[1,4,5,11,16,...|
|  1.0|(23,[1,4,5,11,16,...|
|  1.0|(23,[0,5,10,13,18...|
+-----+--------------------+

None
[Row(label=0.0, features=SparseVector(23, {1: 1.0, 3: 1.0, 8: 1.0, 15: 1.0, 19: 1.0, 22: 1.0}))]


In [142]:
ohe_train_df.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)



In [160]:
ohe_train_rdd = ohe_train_df \
                     .rdd \
                     .cache()

In [161]:
ohe_train_rdd.take(2)

[Row(label=0.0, features=SparseVector(23, {1: 1.0, 3: 1.0, 8: 1.0, 15: 1.0, 19: 1.0, 22: 1.0})),
 Row(label=0.0, features=SparseVector(23, {3: 1.0, 6: 1.0, 10: 1.0, 12: 1.0, 14: 1.0, 15: 1.0}))]

In [162]:
meanDropOut = ohe_train_rdd.map(lambda x: x[0]).mean()
varDropOut = ohe_train_rdd.map(lambda x: x[0]).variance()
print(f"Mean: {meanDropOut}")
print(f"Variance: {varDropOut}")

Mean: 0.5714285714285714
Variance: 0.2448979591836735


In [146]:
v = SparseVector(23, {1: 1.0, 3: 1.0, 8: 1.0, 15: 1.0, 19: 1.0, 22: 1.0})

In [175]:
BASELINE = np.append(meanDropOut, np.zeros(23))

In [165]:
np.dot(v, weight[1:]) 

0.0

In [155]:
np.append(1, v)

array([1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 1., 0., 0., 1.])

In [166]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [156]:
# <--- SOLUTION --->
# part b - OLS gradient descent function
def GradientDescent(trainRDD, testRDD, wInit, nSteps = 20, 
                    learningRate = 0.1, verbose = False):
    """
    Perform nSteps iterations of OLS gradient descent and 
    track loss on a test and train set. Return lists of
    test/train loss and the models themselves.
    """
    # initialize lists to track model performance
    train_history, test_history, model_history = [], [], []
    
    # perform n updates & compute test and train loss after each
    model = wInit
    for idx in range(nSteps):  
        ############## YOUR CODE HERE #############
        model = GDUpdate(trainRDD, model, learningRate)
        training_loss = LogLoss(trainRDD, model) 
        test_loss = LogLoss(testRDD, model)
        ############## (END) YOUR CODE #############
        
        # keep track of test/train loss for plotting
        train_history.append(training_loss)
        test_history.append(test_loss)
        model_history.append(model)
        
        # console output if desired
        if verbose:
            print("----------")
            print(f"STEP: {idx+1}")
            print(f"training loss: {training_loss}")
            print(f"test loss: {test_loss}")
            print(f"Model: {[round(w,3) for w in model]}")
    return train_history, test_history, model_history

In [167]:
# plot error curves - RUN THIS CELL AS IS
def plotErrorCurves(trainLoss, testLoss, title = None):
    """
    Helper function for plotting.
    Args: trainLoss (list of MSE) , testLoss (list of MSE)
    """
    fig, ax = plt.subplots(1,1,figsize = (16,8))
    x = list(range(len(trainLoss)))[1:]
    ax.plot(x, trainLoss[1:], 'k--', label='Training Loss')
    ax.plot(x, testLoss[1:], 'r--', label='Test Loss')
    ax.legend(loc='upper right', fontsize='x-large')
    plt.xlabel('Number of Iterations')
    plt.ylabel('Log loss')
    if title:
        plt.title(title)
    plt.show()

In [177]:
# <--- SOLUTION --->
# part d - write function to compute loss (FILL IN MISSING CODE BELOW)
def LogLoss(dataRDD, W):
    """
    Compute mean squared error.
    Args:
        dataRDD - each record is a tuple of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    """
    augmentedData = dataRDD.map(lambda x: (x[0], np.append([1.0], x[1])))
    ################## YOUR CODE HERE ##################
    loss = augmentedData.map(lambda x: (-x[0] * np.log(sigmoid(W.dot(x[1]))) - (1 - x[0]) * np.log(1 - sigmoid(W.dot(x[1])))) ).mean()
    ################## (END) YOUR CODE ##################
    return loss

In [178]:
# part b - function to perform a single GD step
def GDUpdate(dataRDD, W, learningRate = 0.1):
    """
    Perform one OLS gradient descent step/update.
    Args:
        dataRDD - records are tuples of (features_array, y)
        W       - (array) model coefficients with bias at index 0
    Returns:
        new_model - (array) updated coefficients, bias at index 0
    """
    # add a bias 'feature' of 1 at index 0
    augmentedData = dataRDD.map(lambda x: ( x[0], np.append([1.0], x[1]))).cache()
    
    ################## YOUR CODE HERE ################# 
    grad = augmentedData.map(lambda x: (sigmoid(W.dot(x[1])) - x[0])*x[1]).mean()
    new_model = W - learningRate * grad
    ################## (END) YOUR CODE ################# 
    
    return new_model

In [179]:
ohe_train_rdd \
       .map(lambda x: (x[0], np.append([1.0], x[1]))) \
       .take(2)

[(0.0,
  array([1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 1., 0., 0., 1.])),
 (0.0,
  array([1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
         0., 0., 0., 0., 0., 0., 0.]))]

In [180]:
print(f"Baseline model loss: {LogLoss(ohe_train_rdd, BASELINE)}")

Baseline model loss: [0.5528949  0.69314718 0.69314718 0.69314718 0.69314718 0.69314718
 0.69314718 0.69314718 0.69314718 0.69314718 0.69314718 0.69314718
 0.69314718 0.69314718 0.69314718 0.69314718 0.69314718 0.69314718
 0.69314718 0.69314718 0.69314718 0.69314718 0.69314718 0.69314718]


In [181]:
nSteps = 5
model = BASELINE
print(f"BASELINE:  Loss = {LogLoss(ohe_train_rdd,model)}")
for idx in range(nSteps):
    print("----------")
    print(f"STEP: {idx+1}")
    model = GDUpdate(ohe_train_rdd, model)
    loss = LogLoss(ohe_train_rdd, model)
    print(f"Loss: {loss}")
    print(f"Model: {[round(w,3) for w in model]}")

BASELINE:  Loss = 0.6926036532203024
----------
STEP: 1
Loss: 0.6673236038103592
Model: [0.565, -0.004, 0.001, -0.009, -0.018, 0.01, 0.015, -0.009, 0.005, -0.018, -0.009, -0.008, 0.01, -0.013, 0.005, -0.009, -0.013, 0.01, 0.005, 0.015, -0.009, 0.005, 0.005, -0.009]
----------
STEP: 2
Loss: 0.6435514434651056
Model: [0.558, -0.008, 0.002, -0.018, -0.036, 0.02, 0.03, -0.018, 0.01, -0.036, -0.018, -0.015, 0.02, -0.026, 0.01, -0.018, -0.026, 0.02, 0.01, 0.03, -0.018, 0.01, 0.01, -0.018]
----------
STEP: 3
Loss: 0.6211927776964008
Model: [0.552, -0.011, 0.003, -0.027, -0.053, 0.03, 0.045, -0.027, 0.016, -0.053, -0.027, -0.022, 0.03, -0.038, 0.015, -0.027, -0.038, 0.03, 0.016, 0.045, -0.027, 0.015, 0.016, -0.027]
----------
STEP: 4
Loss: 0.6001561133777915
Model: [0.547, -0.015, 0.004, -0.035, -0.07, 0.039, 0.059, -0.035, 0.021, -0.07, -0.035, -0.029, 0.039, -0.049, 0.02, -0.035, -0.049, 0.039, 0.021, 0.059, -0.035, 0.02, 0.021, -0.035]
----------
STEP: 5
Loss: 0.5803534787891058
Model: [0.5

In [ ]:
LLtrain, LLtest, models = GradientDescent(ohe_train_rdd, testRDD, wInit, nSteps = 50)
print(f"\n... trained {len(models)} iterations in {time.time() - start} seconds")